In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from itertools import product
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from omdc_util import *
from distributions import cmax, smax, sample, expectation, Normal, PointMass, SampleDist, Normal, Categorical

In [2]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    constrain = True
    while constrain:
        dist = [np.random.randint(10,50) for _ in range(attributes)]
        dist = np.around(np.array(dist)/sum(dist),decimals=2)
        constrain = np.min(dist) <= 0.10 or np.max(dist) >= 0.40
    np.random.shuffle(dist)
    return dist

In [3]:
gambles = 7
attributes = 4
high_stakes = Normal((9.99+0.01)/2, 0.3*(9.99-0.01))
low_stakes = Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

train_envs_hd = [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs = train_envs_hd+train_envs_ld 

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [4]:
bo_pol_theta = np.load('data/om_bmps_pols/best/hs_hd_1cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [5]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

def action_coordinate(env, action):
    return (action//env.outcomes,action%env.outcomes)

def p_grid(env, actions):
    grid = np.zeros((env.gambles+1,env.outcomes))
    grid[0,:] = env.dist
    for i in range(len(actions[:-1])):
        gamble, outcome = action_coordinate(env,actions[i]) 
        grid[gamble+1, outcome] = i+1
    return grid

# BMPS Run

In [6]:
train_envs[21].reset()
trace = run_env(bo_pol, train_envs[21])
trace

{'actions': [0, 4, 8, 12, 16, 20, 24, 1, 25, 3, 27, 17, 9, 26, 28],
 'ground_truth': array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
         -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913]),
 'observations': 14,
 'util': 6.2582775286568371}

In [7]:
train_envs[21].dist

array([ 0.36,  0.35,  0.11,  0.18])

In [8]:
train_envs[21].grid()

array([[6.4697168852787925, 6.0874747782387573, Norm(5.00, 2.99), 5.3770647738841024],
       [3.7011480847344491, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.5980424669356585, 0.61757752425761225, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [2.9558591442380684, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.6841451447763056, 3.3036118244326227, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.0261650303765464, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [9.0650927676074815, 3.7866146709433091, 9.3205178230600225, 5.9126224275076842]], dtype=object)

In [9]:
p_grid(train_envs[21],trace['actions'])

array([[  0.36,   0.35,   0.11,   0.18],
       [  1.  ,   8.  ,   0.  ,  10.  ],
       [  2.  ,   0.  ,   0.  ,   0.  ],
       [  3.  ,  13.  ,   0.  ,   0.  ],
       [  4.  ,   0.  ,   0.  ,   0.  ],
       [  5.  ,  12.  ,   0.  ,   0.  ],
       [  6.  ,   0.  ,   0.  ,   0.  ],
       [  7.  ,   9.  ,  14.  ,  11.  ]])

# DC Run

In [10]:
train_envs[21].reset()
trace = run_dc(train_envs[21])
trace

{'actions': [16, 12, 20, 4, 8, 0, 1, 3, 2, 24, 25, 27, 26, 28],
 'ground_truth': array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
         -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913]),
 'observations': 13,
 'options': [(4, 1),
  (3, 1),
  (5, 1),
  (1, 1),
  (2, 1),
  (0, 1),
  (0, 1),
  (0, 1),
  (0, 1),
  (6, 1),
  (6, 1),
  (6, 1),
  (6, 1),
  (-99, 1)],
 'util': 6.2882775286568364}

In [11]:
train_envs[21].dist

array([ 0.36,  0.35,  0.11,  0.18])

In [12]:
train_envs[21].grid()

array([[6.4697168852787925, 6.0874747782387573, -1.1743937774121251, 5.3770647738841024],
       [3.7011480847344491, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.5980424669356585, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [2.9558591442380684, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.6841451447763056, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [4.0261650303765464, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [9.0650927676074815, 3.7866146709433091, 9.3205178230600225, 5.9126224275076842]], dtype=object)

In [13]:
p_grid(train_envs[21],trace['actions'])

array([[  0.36,   0.35,   0.11,   0.18],
       [  6.  ,   7.  ,   9.  ,   8.  ],
       [  4.  ,   0.  ,   0.  ,   0.  ],
       [  5.  ,   0.  ,   0.  ,   0.  ],
       [  2.  ,   0.  ,   0.  ,   0.  ],
       [  1.  ,   0.  ,   0.  ,   0.  ],
       [  3.  ,   0.  ,   0.  ,   0.  ],
       [ 10.  ,  11.  ,  13.  ,  12.  ]])

# Parsing

In [131]:
def make_env(gambles=4, cost=.01, ground_truth=False, dist=hd_dist(3), stakes = 'high'):
    reward = Normal((9.99+0.01)/2, 0.3*(9.99-0.01)) if stakes == 'high' else Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
    return OldMouselabEnv(gambles, dist, reward, cost, ground_truth= ground_truth) 

In [18]:
env = train_envs[21]

In [19]:
env2 = make_env(ground_truth=env.ground_truth,dist=env.dist)

In [20]:
env.ground_truth

array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
        -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913])

In [21]:
env2.ground_truth

array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
        -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913])

In [22]:
trace = run_env(bo_pol, env)
trace

{'actions': [0, 4, 8, 12, 16, 20, 24, 1, 25, 3, 27, 17, 9, 26, 28],
 'ground_truth': array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
         -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913]),
 'observations': 14,
 'util': 6.2582775286568371}

In [23]:
trace = run_env(bo_pol, env2)
trace

{'actions': [0, 4, 8, 12, 16, 20, 24, 1, 25, 3, 27, 17, 9, 26, 28],
 'ground_truth': array([  6.47 ,   6.087,  -1.174,   5.377,   3.701,   1.684,   2.161,  11.175,   4.598,   0.618,   6.018,   2.667,   2.956,  10.258,   6.065,   6.735,   4.684,   3.304,   6.411,   1.559,   4.026,
         -1.827,  12.119,   5.962,   9.065,   3.787,   9.321,   5.913]),
 'observations': 14,
 'util': 6.5382775286568373}

In [172]:
def wrap_po(env,click_sequence,t=1,p_rand=0):
    memo = dict()
    def parse_options_clean(init_state,dist,stakes,pre_acts,click_sequence,t=1,p_err=0.001):
        if click_sequence == []: 
            return True, [[]], [1]
        
        if (tuple(pre_acts),tuple(click_sequence),tuple(dist),t,p_err,stakes) in memo:       
            return memo[(tuple(pre_acts),tuple(click_sequence),tuple(dist),t,p_err,stakes)]   
        
        envc = make_env(ground_truth=init_state, dist=dist, stakes=stakes)
        envc.reset()
        for a in pre_acts:
            envc._step(a)

        option_seqs = []
        likelihoods = []
        done = False
        options, option_insts, option_utils, n_available_clicks,_,_,_ = get_all_options(envc)
        
        for i,j in product(range(1,min(len(dist),len(click_sequence))+1),range(len(options))):
            option = options[j]
            n_insts = len(option_insts[option])
            for inst in option_insts[option]:
                if np.array_equal(click_sequence[:i],inst): 
                    will_done, remaining, rem_likelihoods = (parse_options_clean(init_state,dist,stakes,pre_acts+click_sequence[:i],click_sequence[i:],t,p_rand))
                    done = done or will_done  
                    if done:
                        for k in range(len(remaining)): 
                            option_seqs.append([option]+remaining[k]) 
#                             l_opt_seq = ((1-p_rand)*np.exp(1/t*option_utils[j])/np.sum(np.exp(1/t*option_utils))
#                                         + p_rand*np.prod([1/(n_available_clicks-k) for k in range(option[1])]))
                            alpha = 1 if option == (-1,1) else 0 
                            l_opt_seq = ((1-p_rand)*np.exp(1/t*option_utils[j])/np.sum(np.exp(1/t*option_utils))
                                    + p_rand*alpha)
                            likelihoods.append(l_opt_seq*rem_likelihoods[k]/n_insts)               
        memo[(tuple(pre_acts),tuple(click_sequence),tuple(dist),t,p_err,stakes)] = done, option_seqs, likelihoods
        return done, option_seqs, likelihoods
    stakes = 'high' if env.reward.mu == 5.0 else 'low'
    if click_sequence[-1] != env.term_action:
        click_sequence = click_sequence+[env.term_action]
    return parse_options_clean(env.ground_truth,env.dist,stakes,[],click_sequence,t,p_rand)

In [248]:
# env = train_envs[21]
env = OldMouselabEnv(4, ld_dist(3), reward, cost)
env.reset()
trace = run_dc(env)
trace

{'actions': [0, 1, 2, 6, 9, 3, 4, 7, 8, 10, 11, 12],
 'ground_truth': array([ 8.198,  3.021,  2.501,  4.47 ,  1.334,  5.699,  3.968,  3.67 ,  2.927,  1.122,  8.153,  6.611]),
 'observations': 11,
 'options': [(0, 1),
  (0, 1),
  (0, 1),
  (2, 1),
  (3, 1),
  (1, 1),
  (1, 1),
  (2, 1),
  (2, 1),
  (3, 2),
  (-99, 1)],
 'util': 4.8138536196634396}

In [249]:
a,b,c = wrap_po(env,trace['actions'],t=0.1)

In [250]:
list(zip(list(np.array(b)[np.array(c)!=0]),np.array(c)[np.array(c)!=0]))

[([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 1),
   (1, 1),
   (2, 1),
   (2, 1),
   (3, 1),
   (3, 1),
   (-99, 1)],
  3.1460596600714896e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 1),
   (1, 1),
   (2, 1),
   (2, 1),
   (3, 2),
   (-99, 1)],
  3.9948549976639362e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 1),
   (1, 1),
   (2, 2),
   (3, 1),
   (3, 1),
   (-99, 1)],
  2.6778539354701502e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 1),
   (1, 1),
   (2, 2),
   (3, 2),
   (-99, 1)],
  3.4003290887636501e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 2),
   (2, 1),
   (2, 1),
   (3, 1),
   (3, 1),
   (-99, 1)],
  1.5817839421167515e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 2),
   (2, 1),
   (2, 1),
   (3, 2),
   (-99, 1)],
  2.0085434381896228e-05),
 ([(0, 1),
   (0, 1),
   (0, 1),
   (2, 1),
   (3, 1),
   (1, 2),
   (2, 2),
   (3, 1),
   (3, 1),
   (-9

In [251]:
b[np.argmax(np.array(c))]

[(0, 1),
 (0, 1),
 (0, 1),
 (2, 1),
 (3, 1),
 (1, 1),
 (1, 1),
 (2, 1),
 (2, 1),
 (3, 2),
 (-99, 1)]